In [1]:
from transformer import *

# Parametros

In [2]:
# datos tanto para encoder como decoder
vocab_size = 10000
d_model = 512
num_layers = 6
num_heads = 8
d_ff = 2048
dropout = 0.1
max_seq_length = 10
batch_size = 2

# datos para encoder
tokens = torch.randint(0, vocab_size, (batch_size, max_seq_length))
src_mask = torch.ones(batch_size, 1, max_seq_length, max_seq_length)

# apartir de aqui son datos de prueba para pasar al decoder
def generate_tgt_mask(seq_len):
    return torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0).unsqueeze(0)

tgt_seq_len = max_seq_length
tgt = torch.randint(0, vocab_size, (batch_size, tgt_seq_len))
tgt_mask = generate_tgt_mask(tgt_seq_len)

In [26]:
tokens.dtype

torch.int64

# Encoder Transformer

In [3]:
transformer_encoder = TranformerEncoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_seq_length)

In [4]:
output = transformer_encoder(tokens, src_mask)

In [5]:
print(output.shape)

torch.Size([2, 10, 512])


In [6]:
output

tensor([[[-2.0076e+00,  8.4688e-01, -1.6849e-01,  ..., -1.3073e+00,
           5.4830e-01,  1.1646e+00],
         [-1.1170e+00, -8.4814e-01, -1.0591e+00,  ...,  1.3519e+00,
           3.0030e-01,  1.3062e+00],
         [-8.1709e-01,  1.0031e+00, -1.0164e+00,  ...,  3.5864e-01,
          -2.6820e-02,  2.5098e+00],
         ...,
         [ 1.1815e-01,  1.0719e-02,  8.9530e-01,  ..., -5.6693e-01,
           6.5545e-01, -2.0252e-01],
         [-7.2709e-01,  5.5388e-01, -1.0215e-01,  ..., -9.2923e-01,
           9.9086e-01,  1.1848e+00],
         [-2.3635e+00, -6.5131e-01,  8.4368e-01,  ...,  8.9355e-01,
           3.5279e-01, -3.0267e-02]],

        [[ 7.4488e-02,  5.1456e-01,  3.1903e-01,  ...,  8.3846e-01,
          -4.7906e-01,  7.5857e-01],
         [ 2.1136e-03,  1.3668e+00,  1.2675e+00,  ..., -7.6160e-01,
          -9.0839e-01,  1.7725e+00],
         [-9.1972e-01,  6.6305e-01, -3.5218e-01,  ..., -2.8923e-01,
           1.0341e+00, -8.9025e-01],
         ...,
         [-1.6358e-01, -1

In [6]:
classifier = ClassifierHead(d_model, num_classes=5)

In [7]:
cls_respresentattion = output[:, 0, :]
logits = classifier(cls_respresentattion)
print("Output shape:", logits.shape)
print("Logits:", logits)

Output shape: torch.Size([2, 5])
Logits: tensor([[-1.9659, -2.1014, -1.3732, -1.4973, -1.3445],
        [-2.3736, -1.6140, -1.0863, -1.8565, -1.5414]],
       grad_fn=<LogSoftmaxBackward0>)


In [8]:
regressor = RegressionHead(d_model, output_dim=1)
regression_output = regressor(cls_respresentattion)
print("Regression output shape:", regression_output.shape)

Regression output shape: torch.Size([2, 1])


In [9]:
regression_output

tensor([[-0.5168],
        [-0.2422]], grad_fn=<AddmmBackward0>)

# Decoder Transformer

In [7]:
transformer_decoder = TransformerDecoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_seq_length)

In [8]:
output_decoder = transformer_decoder(tgt, output, tgt_mask)

In [9]:
print(output_decoder.shape)

torch.Size([2, 10, 512])


In [10]:
output_decoder

tensor([[[-1.6325, -0.8952, -0.2896,  ...,  1.8141,  0.8229, -1.2697],
         [-1.4336, -0.8342, -0.5054,  ...,  0.6502,  1.1461, -2.0581],
         [-0.8746,  0.2679,  0.1866,  ..., -0.1587,  0.9529,  0.8123],
         ...,
         [-1.4320,  0.4093,  0.7586,  ...,  0.7835, -0.0945, -0.1435],
         [ 0.0139, -0.0288,  1.0271,  ...,  0.5365,  0.0900, -0.3109],
         [-1.9335,  1.3929,  1.1233,  ..., -0.1698,  0.7826, -0.1913]],

        [[ 0.2891, -0.6804,  0.0572,  ...,  0.6331, -1.2571, -0.9006],
         [ 1.0124, -0.4362,  0.4146,  ...,  0.6686, -0.4035, -1.4881],
         [ 0.5934,  0.7047,  0.7788,  ..., -0.3735, -1.1724,  0.7673],
         ...,
         [ 0.4656,  1.0294,  1.4422,  ...,  2.5686,  0.1262, -1.8301],
         [-0.4930, -1.1110,  1.5603,  ...,  1.1924, -0.0810, -2.0649],
         [-0.9133,  0.8247,  0.8449,  ...,  1.1629, -0.8747,  0.0777]]],
       grad_fn=<NativeLayerNormBackward0>)

regresar probabilidades

In [12]:
class ProjectionLayer(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return self.fc(x)    

projection_layer = ProjectionLayer(d_model, vocab_size)
logits = projection_layer(output_decoder)

In [21]:
print(logits.shape)

torch.Size([2, 10, 10000])


In [15]:
probs = torch.softmax(logits, dim=-1)

In [23]:
probs.shape

torch.Size([2, 10, 10000])

In [25]:
probs[0][0]

tensor([6.6772e-05, 1.0207e-04, 6.7407e-05,  ..., 5.3753e-05, 1.0967e-04,
        7.3463e-05], grad_fn=<SelectBackward0>)

In [19]:
torch.max(probs[0][0])

tensor(0.0009, grad_fn=<MaxBackward1>)

ok, vamos a probar con texto real el transformer